In [1]:
import numpy as np
import torch
import tensorflow as tf
from HelperClasses import Direction, Point, Board, Snake
from SnakeGameClass import SnakeGame
import tensorflow as tf

c:\Users\otto\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


pygame 2.1.2 (SDL 2.0.18, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
game = SnakeGame(10,10)
# game.snake
snake = Snake((10,10))
# print(snake.body)


In [4]:
board = Board(10,10,snake=snake, food=game.food)
board.food
board

     0    1    2    3    4    5    6    7    8    9
0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
2  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
3  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0
4  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
5  0.0  0.0  0.0  0.5  0.5  0.5  0.0  0.0  0.0  0.0
6  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
7  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
8  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
9  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

: 

In [4]:
# board.get_available_space_3directions(Direction.UP)

In [5]:
tens = board.to_tensor()
tens = torch.unsqueeze(tens, 0)
tens.shape


torch.Size([1, 3, 10, 10])

In [6]:
s2 = Snake((10,10))
s2.move(Direction.RIGHT, game.food)
b2 = Board(10,10,1,snake=s2.body, food=game.food)
b2

     0    1    2    3    4    5    6    7    8    9
0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
2  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
3  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
4  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
5  0.0  0.0  0.0  0.0  0.5  0.5  0.5  0.0  0.0  0.0
6  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
7  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
8  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
9  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [7]:
b_t = board.to_tensor(channels=1)
b2_t = b2.to_tensor(channels=1)
b_tf = board.to_tensorflow_tensor()

b_tf_stacked = tf.stack((b_tf,b_tf,b_tf,b_tf), axis=-1)
# b_tf_stacked = tf.stack((b_tf,b_tf), axis=-1)
b_tf_stacked = tf.expand_dims(b_tf_stacked, axis=0)
print(b_t.shape)
print(b2_t.shape)

torch.Size([10, 10])
torch.Size([10, 10])


In [8]:
stacked = torch.stack((b_t,b2_t)).unsqueeze(0)
stacked.shape

torch.Size([1, 2, 10, 10])

In [14]:
channels = 2
net = torch.nn.Sequential(
    torch.nn.Conv2d(channels, 16, (5,5)),
    torch.nn.ReLU(),
    torch.nn.Conv2d(16, 16, (5,5)),
    torch.nn.ReLU(),
    torch.nn.Flatten(),
    torch.nn.Linear(16*2*2,3), # For 10x10 grid
    # torch.nn.Linear(16*3*3,3), # For 20x20 grid
)

tf_model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, 5, activation="relu"),
    tf.keras.layers.Conv2D(32, 3, activation="relu"),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32*4, activation="relu"),
    tf.keras.layers.Dense(4, activation="linear")
    
])




In [15]:
# out = net(tens)
# out = net(stacked)
# tf_model.layers[0](b_tf_stacked)
# out.shape
print(b_tf_stacked.shape)
out = tf_model(b_tf_stacked)
out.shape
# tf.argmax(out[0]).numpy()

(1, 10, 10, 4)


TensorShape([1, 4])

: 

In [11]:
temp = torch.stack((stacked[0],stacked[0]))

temp.shape


net(temp)

tensor([[-0.0742,  0.0941,  0.0351],
        [-0.0742,  0.0941,  0.0351]], grad_fn=<AddmmBackward0>)